In [7]:
# model, Gurobi
# unrelated parallel machine scheduling
from gurobipy import *
import pandas as pd

# read the excel data into dataFrame
data = pd.read_excel("source_data.xlsx",index_col = 0, names=None )   # headers = None,sheet_name='setuptime',use_cols =
ptList = data.values.tolist()

data = pd.read_excel("source_data.xlsx",index_col = 0, names=None,sheet_name='setuptime')   # headers = None,sheet_name='setuptime',use_cols =
setupList = data.values.tolist()


number_job = len(ptList)
number_machine = len(ptList[1])
processing_time = ptList
iSet = range(number_machine)
kSet = range(1, number_job+1)
jSet = range(number_job+1)

# range in python is [1, 2), including 1, not including 2 
# print(iSet)
# print(jSet)
# print(kSet)


# Initial the gurobi model
m = Model()

#add variables 
# if job j precedes job k on machine i
x = m.addVars(number_machine,jSet, kSet, vtype=GRB.BINARY, name='x')
# S = m.addVars(number_machine,jSet, kSet, vtype=GRB.BINARY, name='S')
C = m.addVars(number_machine,jSet, vtype=GRB.CONTINUOUS, name='C')
Cmax = m.addVar(lb = 0, name='makespan')

# Set the obj which means the minmize the makespan
m.setObjective(Cmax,GRB.MINIMIZE)


# it's inspiring to use a dummy 0 in Job set J, useful
# add constraints
for k in (kSet):  # pay attention to in and in rnage()
    m.addConstr(quicksum(x[i,j,k] for j in (jSet) if j != k for i in iSet) == 1) 

for j in (kSet):
    m.addConstr(quicksum(x[i,j,k] for k in (kSet) if j != k for i in iSet) <= 1) 

for i in (iSet):
    m.addConstr(quicksum(x[i,0,k] for k in (kSet)) <= 1) 

for i in (iSet):
    for j in (kSet):
        for k in (kSet):
            if( k !=j ):    
                m.addConstr(quicksum(x[i,h,j] for h in (jSet) if h != j if h!=k) >= x[i,j,k])
for i in (iSet):
    for j in (jSet):
        for k in (kSet):
            if(k != j):
                m.addConstr(C[i,k] + 1000*(1-x[i,j,k]) >= C[i,j]+processing_time[k-1][i]+setupList[j][k])

for i in (iSet):
    m.addConstr(C[i,0] ==0) 

for i in (iSet):
    for k in kSet:
        m.addConstr(C[i,k] <= Cmax) 

m.optimize()




# Get the solution
# construct the 2-d array to store the jobs processed on the different machines
machine_list_j =[]
machine_list_k =[]
machine_list = []
for i in range(number_machine):
    machine_list.append([])
    machine_list_j.append([])
    machine_list_k.append([])
    
for i in (iSet):
    for j in (jSet):
        for k in (kSet):
            if(x[i,j,k].X == 1):
                machine_list_j[i].append(j)
                machine_list_k[i].append(k)

for i in (iSet):
    k = machine_list_k[i][machine_list_j[i].index(0)]
    machine_list[i].append(0)
    
    while(k):
        machine_list[i].append(k)
        if k in machine_list_j[i]:
            k = machine_list_k[i][machine_list_j[i].index(k)]
        else:
            k=0
    
    print(machine_list[i])

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 626 rows, 364 columns and 4263 nonzeros
Model fingerprint: 0xfa9cc592
Variable types: 34 continuous, 330 integer (330 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 31.0000000
Presolve removed 3 rows and 33 columns
Presolve time: 0.01s
Presolved: 623 rows, 331 columns, 4230 nonzeros
Variable types: 31 continuous, 300 integer (300 binary)

Root relaxation: objective 0.000000e+00, 207 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   27   31.00000    0.00000   100%     -    0s
H    0     0                      30.0000000    0.00

ValueError: 0 is not in list

In [8]:
m.printAttr('X')


    Variable            X 
-------------------------
   x[0,0,10]            1 
    x[0,7,9]            1 
   x[0,10,7]            1 
    x[1,0,5]            1 
    x[1,5,6]            1 
    x[1,6,3]            1 
    x[2,0,1]     0.999997 
    x[2,0,4]  3.01597e-06 
    x[2,1,2]            1 
    x[2,2,4]     0.999997 
    x[2,4,8]            1 
    x[2,8,1]   3.0148e-06 
      C[0,2]            4 
      C[0,3]            9 
      C[0,4]            9 
      C[0,5]      6.02676 
      C[0,6]           15 
      C[0,7]      9.99997 
      C[0,8]            8 
      C[0,9]           15 
     C[0,10]            4 
      C[1,1]      6.02676 
      C[1,2]           15 
      C[1,3]           12 
      C[1,4]      5.00001 
      C[1,5]            3 
      C[1,6]            9 
      C[1,9]            5 
     C[1,10]      6.02676 
      C[2,1]      2.00001 
      C[2,2]      6.00001 
      C[2,3]            3 
      C[2,4]           11 
      C[2,5]      6.02676 
      C[2,6]            9 
 

In [4]:
# # get the start time and completion time for each job
# print setup time
# 2-demension list

m_time = [0] *number_machine
start =[0]  * (number_job+1)
finish =[0] * (number_job+1)
for i in range(len(machine_list)):
    for j in range(1, len(machine_list[i])):
        start[machine_list[i][j]] = m_time[i]+setupList[machine_list[i][j-1]][machine_list[i][j]]
        finish[machine_list[i][j]] = start[machine_list[i][j]]+processing_time[machine_list[i][j]-1][i]
        m_time[i] = finish[machine_list[i][j]]

start_s = [[]] *number_machine
finish_s = [[]] *number_machine
for i in range(number_machine):
    for j in range(len(machine_list[i])-1):
        start_s[i].append(finish[machine_list[i][j]])
        finish_s[i].append(start_s[i][j]+setupList[machine_list[i][j]][machine_list[i][j+1]])

    
# Print the results （absolute time）
print (start)
print (finish)
print (start_s)
print (finish_s)

[0, 1, 3, 7, 1, 13, 10, 4, 4, 10, 0]
[0, 2, 6, 9, 4, 15, 15, 9, 13, 14, 4]
[[0, 4, 9, 0, 4, 13, 0, 2, 6, 9], [0, 4, 9, 0, 4, 13, 0, 2, 6, 9], [0, 4, 9, 0, 4, 13, 0, 2, 6, 9]]
[[0, 4, 10, 1, 4, 9, 1, 5, 10, 1], [0, 4, 10, 1, 4, 9, 1, 5, 10, 1], [0, 4, 10, 1, 4, 9, 1, 5, 10, 1]]


In [6]:
i = 2
j =3
print(setupList[machine_list[i][j]][machine_list[i][j+1]])

1


In [ ]:
from datetime import datetime  
from datetime import timedelta

import plotly.express as px

# organize the data (absolute date time) to plotly
# suppose we use 2020-01-01 as the baseline time
start_date=[]
finish_date =[]

start_date_base = datetime.strptime('2020-01-01','%Y-%m-%d')

for i in range(number_job+1):
    start_date.append((start_date_base + timedelta(days=start[i])).strftime('%Y-%m-%d'))
    finish_date.append((start_date_base + timedelta(days=finish[i])).strftime('%Y-%m-%d'))

row_list = []
Job_index =0
for i in range(len(machine_list)):
    for j in range(len(machine_list[i])):
        dict1 = dict(Machine = "Machine"+str(i+1), Job  = "Job"+str(machine_list[i][j]+1), Start = start_date[machine_list[i][j]], Finish = finish_date[machine_list[i][j]])
        row_list.append(dict1)

df = pd.DataFrame(row_list)
print (df)

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Machine", color = 'Job')
fig.update_layout(showlegend=False) 
fig.show()